In [6]:
from jupyter_plotly_dash import JupyterDash

from jupyter_dash import JupyterDash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

from AnimalShelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "password"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),
             alt='customer image', style={'height':'5%', 'width':'5%'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.H2('Scott Laboe')),
    html.Hr(),
    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
               {'label': 'Water Rescue', 'value': 1},
               {'label': 'Mountain Rescue', 'value': 2},
               {'label': 'Disaster Rescue', 'value': 3},
               {'label': 'All','value': 4}
            ],
            value='MTL',
            labelStyle={'display': 'inline-block'}
        ) 
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        fixed_columns={'headers': True, 'data': 1},
        style_table={'minWidth': '100%'},
        data=df.to_dict('records'),
        sort_action='native',
        page_size=10
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
 
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(value):
    
    if (value==1):
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog",
        "breed":{"$in":["Labrador Retriever Mix","Chesa Bay Retr Mix","Newfoundland Mix"]},
        "sex_upon_outcome":"Intact Female",
        "age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}})))
    elif (value==2):
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog",
        "breed":{"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},
        "sex_upon_outcome":"Intact Male",
        "age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}})))
    elif (value==3):
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog",
        "breed":{"$in":["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]},
        "sex_upon_outcome":"Intact Male",
        "age_upon_outcome_in_weeks":{"$gte":20},"age_upon_outcome_in_weeks":{"$lte":300}})))
    else:
        df = pd.DataFrame.from_records(shelter.read({}))
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')

    return (data,columns)


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    if viewData is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(
            figure = px.pie(dff, names = 'breed')
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    if viewData is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    loc = [dff.location_lat[0], dff.location_long[0]]
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=loc, zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=loc, children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1('Animal Name'),
                    html.P(dff.name[0])
                ])
            ])
        ])
    ]



app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/
